In [1]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

# 0 -Lire les données dans le dataset

In [2]:
train_url="http://bit.ly/titanic-train-set"
test_url="http://bit.ly/titanic-test-set"

# 1- Récuperer les données

In [3]:
df_train=pd.read_csv(train_url,index_col="PassengerId")
df_test=pd.read_csv(test_url,index_col="PassengerId")

# 2-Exploration des données -- Exploratory Data Analysis (EDA) 
Sert à repondre aux questions ci dessous

regarde dans la competion kaggle certaines personnes ont fait des EDA très 
intéressantes
## Les valeurs manquantes dans le training et test set
## Les colonnes categoriques
Etapes particulières pour les gérer
## Parmi les colonnes il y a du texte, Est ce que certains sont utilisables?
c'est applicable aussi aux dates
## Est ce qu'il y a des colonnes qu'on veut supprimer ou laisser passer telle quelle 

## Les valeurs manquantes dans le training et test set

#### Question bonus pour améliorer les performances
- Est ce qu'il y a des valeurs extremes qui risquent de perturber nos algos ( gestion de outliers) OUTLIERS
- est ce que je désire changer la plage des valeurs de mes colonnes ( standart scaler / Robust scaler / log transforme ) TRANSFORM
- Est ce que je souhaite réduire le nombre de colennes -- FEATURES ENGINEERING / DIMENSIONALITE REDUCTION

In [4]:
df_train.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [5]:
# les valeurs manquantes par colonnes
df_train.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [7]:
df_train.head(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [8]:
target=['Survived']


# Soit on jette , soit on laisse passer
drop_cols=['Ticket'] # liste des colonnes à suprimer
passthrough_cols=['Pclass','SibSp','Parch'] # liste des colonnes à laisser passer

# Avec valeurs manquantes et/ou catégoriques et/ou text
# ---- colonne ou on va devoir faire du processing (Feature Engineering, data cleaning)
numérique_avec_valeurs_manquantes  =['Age','Fare']
catégorique_avec_valeurs_manquantes=['Embarked']
catégorique_sans_valeurs_manquantes=['Sex']
text_cols=['Cabin','Name']


# Verifier si on mis une colonne deux fois
# Vérifier si on a oublié une colonne
# Vérifier si on a fait une faute dans le nom d'une colonne

In [9]:
liste_all_listes=[
target,
drop_cols,
passthrough_cols,
numérique_avec_valeurs_manquantes ,
catégorique_avec_valeurs_manquantes,
catégorique_sans_valeurs_manquantes,
text_cols,
]
liste_all_listes

[['Survived'],
 ['Ticket'],
 ['Pclass', 'SibSp', 'Parch'],
 ['Age', 'Fare'],
 ['Embarked'],
 ['Sex'],
 ['Cabin', 'Name']]

# Vérifier le travail sur nos colonnes

In [10]:
set_cols=set()
for liste in liste_all_listes:
  # liste par liste
  for col in liste: # je verifie que j'ai déjà mis col
    if col in set_cols:  # si j'ai dejà mis affiche moi warning
      print("Warning ;",col," est déjà mise")
    set_cols.add(col)# j'ajoute 



set_column_originale=set(df_train.columns)
mal_ecrites= set_cols - set_column_originale
col_manquantes=set_column_originale - set_cols
print("Si tous est bon, affiche des sets vides")
print("---------------------------------")
print('colonnes mal ecrites',mal_ecrites)
print('colonnes manquantes',col_manquantes)

Si tous est bon, affiche des sets vides
---------------------------------
colonnes mal ecrites set()
colonnes manquantes set()


# Etape - Séparer les features X de la variable cible

In [11]:
target

['Survived']

In [12]:
X=df_train.drop('Survived',axis='columns')


In [13]:
Y=df_train['Survived']


In [14]:
X.shape , Y.shape

((891, 10), (891,))

# Tache de Classification /Tache de Régression
- une tache de classification est lorsque la variable cible est catégorique 
- une tache de classification est lorsque la variable cible est numérique 

# Preprocessing

In [15]:
df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
## Importer les outils pour le preprocessing en tant que tel
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer # gere les objets 1D
from sklearn.preprocessing import FunctionTransformer

In [17]:
## Importer les outils qui nous premettent de faire ça clean
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


In [23]:
# Pipeline de Preprocessing
cat_manquante_pipeline=make_pipeline(
       SimpleImputer(strategy='most_frequent'),
       OneHotEncoder(sparse=False,handle_unknown='ignore')
)

import pandas as pd
def extraire_la_premiere_lettre(serie):
  # récuperer une serie en argument
  # retourner un DataFrame (Compatible)
  return (pd.DataFrame(serie.str[0]))# recuperer la première lettre 

preprocess_cabin=make_pipeline(
          FunctionTransformer(extraire_la_premiere_lettre),
          SimpleImputer(strategy='constant', fill_value='MANQUANTE'),
          OneHotEncoder(handle_unknown='ignore')
)


In [24]:
preprocess_cabin.fit_transform(X['Cabin'])

<891x9 sparse matrix of type '<class 'numpy.float64'>'
	with 891 stored elements in Compressed Sparse Row format>

In [25]:
preprocessing=make_column_transformer(
           ( OneHotEncoder(handle_unknown='ignore') , catégorique_sans_valeurs_manquantes),
           ( cat_manquante_pipeline                 , catégorique_avec_valeurs_manquantes),
           ( SimpleImputer(strategy='median')       , numérique_avec_valeurs_manquantes),
           ( CountVectorizer()                      , 'Name'),
           ( preprocess_cabin                       , 'Cabin'),
           ( 'passthrough'                          , passthrough_cols),
           ( 'drop'                                 , drop_cols)
)

In [26]:
# test ! et ça marche
preprocessing.fit_transform(X)

<891x1528 sparse matrix of type '<class 'numpy.float64'>'
	with 9393 stored elements in Compressed Sparse Row format>

# Machine  Learning (ML)
## Creation des pipeline de ML

In [27]:
from sklearn.pipeline import Pipeline

In [57]:
# Importer les algos qu'on veut tester
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC


In [33]:
knn_pipe=Pipeline([
                  ( 'preprocessing', preprocessing ),
                  ( 'knn', KNN() )
])

In [35]:
# Entrainer la meilleur pipeline sur tout le dataset disponible
knn_pipe.fit(X,Y)

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True),
                                                  ['Sex']),
                                                 ('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimp

In [37]:
knn_pipe.predict(df_test)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [52]:
svc_pipeline=Pipeline([
                   ( 'preprocessing',preprocessing ),
                   ( 'svc', SVC() ),
])

In [53]:
svc_pipeline.fit(X,Y)

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True),
                                                  ['Sex']),
                                                 ('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimp

In [54]:
svc_pipeline.predict(df_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [71]:
ridge_pipe=Pipeline([
                     
                    ('preprocessing',preprocessing),
                     ('ridge',RidgeClassifier()),
])

In [72]:
ridge_pipe.fit(X,Y)

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True),
                                                  ['Sex']),
                                                 ('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimp

# Mesure de performance du modèle
la métrique la plus connue est l'accuracy, il y a une autre métrique qui est la précision
- l'accuracy est le pourcentage de prédiction correcte

In [73]:
prediction_knn=knn_pipe.predict(X)
prediction_rf=rf_pipeline.predict(X)
prediction_svc=svc_pipeline.predict(X)
prediction_ridge=ridge_pipe.predict(X)

In [68]:
def accuracy(pred,verite):
  return ((pred==verite).mean())

In [69]:
accuracy(prediction_knn,Y)

0.8249158249158249

In [74]:
accuracy(prediction_ridge,Y)

0.9988776655443322

# K-fold Cross Validation

In [75]:
from sklearn.model_selection import KFold

In [77]:
cross_val_folds=KFold(n_splits=5,
                      shuffle=True,
                      random_state=777)

In [78]:
from sklearn.model_selection import cross_val_score


# Gridsearch CV

In [95]:
from sklearn.model_selection import GridSearchCV

In [98]:
knn={}
# pipeline KNN
knn['pipeline']=Pipeline([
                  ('preprocessing',preprocessing),
                   ('knn',KNN())
])
# grilles pour la pipeline KNN
knn['hyperparametres']={}
knn['hyperparametres']['knn__n_neighbors']=[1,3,5,7,13,17,21,27,29]
knn['hyperparametres']['knn__weights']=['uniform','distance']

# gridsearch pour la pipeline KNN
knn['gridsearch']=GridSearchCV(estimator    = knn['pipeline'],
                               param_grid   = knn['hyperparametres'],
                               scoring      ='balanced_accuracy',
                               cv           = cross_val_folds)

In [100]:
knn['gridsearch'].fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                              

In [102]:
knn['gridsearch'].best_params_

{'knn__n_neighbors': 3, 'knn__weights': 'distance'}

In [103]:
knn['gridsearch'].best_score_

0.7112541982539469

In [110]:
ridge={}
# pipeline ridge
ridge['pipeline']=Pipeline([
                  ('preprocessing',preprocessing),
                   ('ridge',RidgeClassifier())
])
# grilles pour la pipeline ridge
ridge['hyperparametres']={}
ridge['hyperparametres']['ridge__alpha']=[1.9,2.3,2.7,3,3.3,4,5,6,7,9.3,9.5,9.7,10,10.5]

# gridsearch pour la pipeline ridge
ridge['gridsearch']=GridSearchCV(estimator    = ridge['pipeline'],
                               param_grid   = ridge['hyperparametres'],
                               scoring      ='balanced_accuracy',
                               cv           = cross_val_folds)

In [111]:
ridge['gridsearch'].fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                              

In [112]:
ridge['gridsearch'].best_params_

{'ridge__alpha': 9.3}

In [113]:
ridge['gridsearch'].best_score_

0.8086833054813573

In [135]:
rf={}
# pipeline rf
rf['pipeline']=Pipeline([
                  ('preprocessing',preprocessing),
                   ('rf',RandomForest())
])
# grilles pour la pipeline rf
rf['hyperparametres']={}
rf['hyperparametres']['rf__n_estimators']=[10,25,100,150,200]
rf['hyperparametres']['rf__max_features']=['auto','log2','sqrt',None,0.1,0.3,0.6,0.85]
rf['hyperparametres']['rf__max_depth']=[None,2,6,10,15]
rf['hyperparametres']['rf__min_samples_split']=[2,5,10]

# gridsearch pour la pipeline rf
rf['gridsearch']=GridSearchCV(estimator    = rf['pipeline'],
                               param_grid   = rf['hyperparametres'],
                               scoring      ='balanced_accuracy',
                               cv           = cross_val_folds)

In [136]:
rf['gridsearch'].fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                              

In [137]:
rf['gridsearch'].best_params_

{'rf__max_depth': 10,
 'rf__max_features': 0.3,
 'rf__min_samples_split': 5,
 'rf__n_estimators': 25}

In [139]:
rf['gridsearch'].best_score_, ridge['gridsearch'].best_score_,knn['gridsearch'].best_score_

(0.8169623858814923, 0.8086833054813573, 0.7112541982539469)

In [143]:
# Le modèle finale

pipeline_final=make_pipeline(preprocessing, RandomForest(max_depth=10,
                                                      max_features=0.3,
                                                      min_samples_split=5,
                                                      n_estimators=25)
)
                          

In [144]:
pipeline_final.fit(X,Y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True),
                                                  ['Sex']),
                                                 ('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpl

In [145]:
pipeline_final.predict(df_test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,